# Tile JPG, PNG, TIF, TIFF image formats into 224x224 squares, optionally with specified GeoJSON AOI 

## Created by Julian Lee, April 8 2021

In [2]:
## Import Packages - if not installed, use conda or pip to install

import os
import rasterio as rio
import shapely.geometry
import datetime
import utm
import numpy as np

from pathlib import Path
from itertools import product
from rasterio import windows
from rasterio.mask import mask, raster_geometry_mask

#### User-defined parameters

In [3]:
image_source = 'Planet' # <--- Sentinel-1 or Planet
image_name = '20210316_141406_103b_3B_AnalyticMS.tif' # < --- Image file name

## Tile width & height
tile_width, tile_height = 224, 224

## Define overlap of tile windows in px
width_overlap, height_overlap = 20, 20

## Directory path of source image; modify to your own
in_path = './images/' + image_source + '/originals/' + image_name

## Relative directory path to save tiles; modify to your own
out_path = './images/' + image_source + '/tiles/' + datetime.date.today().isoformat() + '/'

## Optional GeoJSON Feature coordinates of area within image to tile, if image is georeferenced (.tif or .tiff)
geojson = None

#### Tiling script

In [ ]:
## If output path doesn't exist, create it
Path(out_path).mkdir(parents=True, exist_ok=True)

def get_tiles(ds, crop=None, width=224, height=224):
    
    ## Define crop window offsets with preset overlap
    if crop:
        nols, nrows = crop.width, crop.height
        offsets = product(range(crop.col_off, crop.col_off + nols, width - width_overlap), range(crop.row_off, crop.row_off + nrows, height - height_overlap))
    else:
        nols, nrows = ds.width, ds.height
        offsets = product(range(0, nols, width - width_overlap), range(0, nrows, height - height_overlap))
    
    big_window = windows.Window(col_off=0, row_off=0, width = ds.width, height = ds.height)
    
    for col_off, row_off in offsets:
        
        ## Handle edge cases to maintain dimensions
        if col_off + width > ds.width: 
            col_off = ds.width - width
        if row_off + height > ds.height: 
            row_off = ds.height - height
            
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
        
with rio.open(in_path) as inds:
    crop_window = None
    
    if inds.driver == 'GTiff':
        output_filename = 'tile_{}-{}.tif'
    elif inds.driver == 'PNG':
        output_filename = 'tile_{}-{}.png'
    else:
        output_filename = 'tile_{}-{}.jpg'
    
    ## If GeoJSON AOI provided and image type is .tif or .tiff, crop the image
    if geojson and inds.driver == 'GTiff':
        
        ## If image georeferencing is in lat-long, directly crop with the GeoJSON
        if np.abs(inds.bounds.left <= 180) and np.abs(inds.bounds.top < 90): 
            geom = shapely.geometry.Polygon(geojson['geometry']['coordinates'][0])
       
        ## If image georeferencing is in UTM, convert GeoJSON to UTM before cropping
        else:
            for idx, pair in enumerate(geojson['geometry']['coordinates'][0]):
                x, y, _, _ = utm.from_latlon(pair[1], pair[0])
                geojson['geometry']['coordinates'][0][idx][0] = x
                geojson['geometry']['coordinates'][0][idx][1] = y
            geom = shapely.geometry.Polygon(geojson['geometry']['coordinates'][0])
        
        crop_window = raster_geometry_mask(inds, [geom], crop=True)[2]
    
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds, crop_window):
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
        with rio.open(outpath, 'w', **meta) as outds:
            outds.write(inds.read(window=window))